In [ ]:
import os
os.chdir("..")

In [ ]:
from CensusForge import CensusAPI
import polars as pl
import requests
import pandas as pd
capi = CensusAPI()

In [ ]:
empty_df = [
    pl.Series("date", dtype=pl.Datetime),
    pl.Series("census_value", dtype=pl.Int64),
    pl.Series("comm_level", dtype=pl.String),
    pl.Series("naics_code", dtype=pl.String),
    pl.Series("country_name", dtype=pl.String),
    pl.Series("contry_code", dtype=pl.String),
]
census_df = pl.DataFrame(empty_df)
dataset = "/acs/acs5/profile"
params = [
    "DP03_0051E",
    "DP03_0052E",
    "DP03_0053E",
    "DP03_0054E",
]
year = 2019
url = f"https://api.census.gov/data/{year}/{dataset}?get={params}"

In [ ]:
capi.get_database_id(name="acs-acs5-profile")

In [ ]:

capi.get_variable_id(name="DP03_0051E")

In [ ]:
result = capi.conn.execute(
    """
    SELECT 
        GROUP_CONCAT(DISTINCT y.year) AS available_years
    FROM sqlite_db.variable_interm AS v
    INNER JOIN sqlite_db.year_table AS y 
        ON v.year_id = y.id
    WHERE v.dataset_id = 26;
    """
).fetchall()

year_list = list(map(int, result[0][0].split(','))) if result[0][0] else []
sorted(year_list)

In [ ]:
df = capi.conn.execute(
    """
    SELECT *
        FROM sqlite_db.variable_interm 
        WHERE dataset_id=26 AND var_id=59047;
    """
).df()

In [ ]:
df[["year_id", "var_id"]]
matrix = pd.crosstab(df['var_id'], df['year_id'])
matrix = (matrix > 0).astype(int)
matrix

In [ ]:
capi.conn.execute(
    """
    SELECT * FROM sqlite_db.dataset_table AS d
    INNER JOIN sqlite_db.variable_interm AS v 
        ON d.id = v.dataset_id
    WHERE d.dataset = ?;
    """, 
    ("acs-acs5-profile",)
).df()

In [ ]:
capi.conn.execute(
    """
    SELECT 
        GROUP_CONCAT(DISTINCT y.year) AS available_years
    FROM sqlite_db.variable_interm AS v
    INNER JOIN sqlite_db.year_table AS y 
        ON v.year_id = y.id
    WHERE v.dataset_id = 26 
    AND v.var_id = 59047;
    """
).df()

In [ ]:
capi.conn.execute(
    """
    SELECT * FROM sqlite_db.year_table;
    """
).df().head()